In [37]:
import qiskit 


In [38]:

# Importing standard Qiskit libraries
from qiskit import *
from qiskit.visualization import *
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import math

In [39]:
def adder(qc, a, b, c, m):
    qc.reset(c)
    
    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) 
    #with output fed to c[i+1]
    for i in range(m - 1):
        # print(i)
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i+1])
    
    #For the last iteration of the carry gate, instead of feeding the
    #result to c[n], we use b[n], which is why c has only n bits,
    #with c[n-1] being the last carry bit
    qc.ccx(a[m - 1], b[m - 1], b[m])
    qc.cx(a[m-1], b[m-1])
    qc.ccx(c[m-1], b[m-1], b[m])
    
    #Reversing the gate operation performed on b[n-1]
    qc.cx(a[m - 1], b[m - 1])
    
    #
    qc.cx(a[m - 1], b[m - 1])
    qc.cx(c[m - 1], b[m - 1])
    
    for i in range(m - 2, -1, -1):
        #Reversing the gate operations performed during the carry gate
        #implementations, which is done to reset all carry bits to 
        #the |0> state
        qc.ccx(c[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(a[i], b[i], c[i + 1])
        
        #These two operations act as a sum gate; if a control bit is 
        #in the |1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(a[i], b[i])
        qc.cx(c[i], b[i])
    qc.barrier()


In [40]:
# addInverse(mod, b)
def inverseAdder(qc, a, b, c, m):
   
    qc.reset(c)
    
    for i in range(m - 1):
        #These two operations act as a sum gate; if a control bit is 
        #in the |1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(c[i], b[i])
        qc.cx(a[i], b[i])
        
        #Reversing the gate operations performed during the carry gate
        #implementations, which is done to reset all carry bits to 
        #the |0> state
        qc.ccx(a[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i + 1])
    
    #
    qc.cx(c[m - 1], b[m - 1])
    qc.cx(a[m - 1], b[m - 1])
    
    #Reversing the gate operation performed on b[n-1]
    qc.cx(a[m - 1], b[m - 1])
    
    #For the last iteration of the carry gate, instead of feeding the
    #result to c[n], we use b[n], which is why c has only n bits,
    #with c[n-1] being the last carry bit
    qc.ccx(c[m-1], b[m-1], b[m])
    qc.cx(a[m-1], b[m-1])
    qc.ccx(a[m - 1], b[m - 1], b[m])
        
        
    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) 
    #with output fed to c[i+1]
    for i in range(m - 2, -1, -1):
        # print(i)
        qc.ccx(c[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(a[i], b[i], c[i+1])
    qc.barrier()


In [41]:
def assign(qc, q, c):
    qc.reset(q)
    
    # Initializing |a> qubits according to input values
    i = 0
    while(c != 0):
        if(c % 2 == 1):
            qc.x(q[i])
            # print(1)
        c = math.floor(c/2)
        # print(x)
        i = i + 1


def sizeReq(a):
    i = 0
    while(a != 0):
        a = math.floor(a/2)
        i = i + 1
    
    return i

def quantumAnd(qc, a0, a1, ancillary):
    qc.ccx(a0, a1, ancillary)
    qc.barrier()

In [42]:
def modularAdder(qc : QuantumCircuit, a, b, c, mod, temp, flag, m): 
   
    # modular addition begins here
    adder(qc, a, b, c, m)
    
    # this while loop is not working
    # have to remove it 
    with qc.while_loop((flag, 0)):
        qc.reset(temp)
        inverseAdder(qc, mod, b, c, m)
    
        qc.cx(b[m], temp)
        qc.measure(temp, flag)
    
    qc.reset(temp)
    qc.measure(temp, flag)
    
    # adding mod to the negative number
    adder(qc, mod, b, c, m)  
    qc.barrier()  

def modularAdderWithZero(qc : QuantumCircuit, b, c, mod, temp, flag, m):
    # this while loop is not working
    # have to remove it 

    with qc.while_loop((flag, 0)):
        qc.reset(temp)
        inverseAdder(qc, mod, b, c, m)
    
        qc.cx(b[m], temp)
        qc.measure(temp, flag)
    
    qc.reset(temp)
    qc.measure(temp, flag)
    
    # adding mod to the negative number
    adder(qc, mod, b, c, m)   

In [43]:
# (first * second) % N
# a * x MOD N
def modularMultiplier(qc: QuantumCircuit, a, x, control, product, c, MOD, temp, flag, ancillary, bitAncillary,  firstLen, second, m):
    
    # modular multiplication without using any classical bit for conditional statements
    for i in range(2):
        
        quantumAnd(qc, x[i], control, ancillary)

        # golden block
        shiftedA = second
        j = 0
        # if(i + j >= m):
        #     print('breaking')
        #     break

        while(shiftedA != 0):
            if(shiftedA % 2 == 1):
                qc.x(bitAncillary)

            print('j = ', j)
            print('i = ', i)
            print('i + j = ', (i + j))
            quantumAnd(qc, ancillary, bitAncillary, a[j + i])

            if(shiftedA % 2 == 1):
                qc.x(bitAncillary)
                
            shiftedA = math.floor(shiftedA/2)
            j = j + 1
        
        # preparing (a * 2 ^ k) mod N
        modularAdderWithZero(qc, a, c, MOD, temp, flag, m)
        
        # modular addition
        modularAdder(qc, a, product, c, MOD, temp, flag, m)

        qc.reset(a)
        
        quantumAnd(qc, x[i], control, ancillary)

    # if x = 0 then product should be a 
    qc.x(control)

    shiftedA = second
    j = 0
    while(shiftedA != 0):
        if(shiftedA % 2 == 1):
            qc.x(a[j])
        
        qc.ccx(a[j], control, product[j])
            
        shiftedA = math.floor(shiftedA/2)
        j = j + 1

    qc.x(control)

In [44]:
def inverseModularAdder(qc: QuantumCircuit, a, product, m): 

    for i in range(m + 1):
        qc.swap(a[i], product[i])
    
    qc.reset(product)
    

In [48]:
def modularExponentiation(A, X, N):
    
    lenA = sizeReq(A) # x
    lenX = sizeReq(X) # a
    lenN = sizeReq(N) # n
    m = max(lenN, lenA * lenX)

    print('m = ', m)
    
    # declaring required registers
    a = QuantumRegister(m + 1, 'a') #first number
    x = QuantumRegister(m, 'x') #second number
    n = QuantumRegister(m, 'n') #N 
    ans = QuantumRegister(m + 1, 'ans') #Product 
    one = QuantumRegister(m + 1, 'one') #One
    c = QuantumRegister(m, 'c') #Carry bits
        
    cla = ClassicalRegister(m + 1, 'cla') #first number
    clb = ClassicalRegister(m, 'clb') #second number
    clans = ClassicalRegister(m + 1, 'clans') #Final output
    cln = ClassicalRegister(m + 1, 'cln') #n
    clone = ClassicalRegister(m + 1, 'clone') #ans
    
    # conditional qubits required for modularAddition
    temp = QuantumRegister(1, 'temp') 
    control = QuantumRegister(1, 'control')
    ancillary = QuantumRegister(1, 'ancillary')
    bitAncillary = QuantumRegister(1, 'bitAncillary')
    flag = ClassicalRegister(1, 'flag') 

    #Combining all of them into one quantum circuit
    qc = QuantumCircuit(control, ancillary, bitAncillary, a, x, ans, c, n, one, temp, cla, clb, clans, cln, clone, flag)
    
    # initialization of qubits
    # assign(qc, a, A)
    assign(qc, x, X)
    assign(qc, n, N)
    assign(qc, ans, 0)
    assign(qc, one, 1)
    
    qc.barrier()

    # modular exponentiation starts here
    pre = A
    for i in range(lenX):
        
        # control = x[i]
        # assign(qc, a, pre)
        qc.reset(ancillary)
        qc.reset(bitAncillary)
        qc.reset(temp)

        modularMultiplier(qc, a, one, x[i], ans, c, n, temp, flag, ancillary, bitAncillary, m, pre, m)

        inverseModularAdder(qc, one, ans, m)
        
        pre = pre * pre     
    

    # Measuring all the qubits
    for i in range(m + 1):
        qc.measure(one[i], clone[i])
    for i in range(m + 1):
        qc.measure(ans[i], clans[i])
    for i in range(m):
        qc.measure(a[i], cla[i])
    for i in range(m):
        qc.measure(x[i], clb[i])
    for i in range(m):
        qc.measure(n[i], cln[i])
    
    qc.measure(control, flag)
    print(qc)
    
    #chosing backend and executing job
    backend = AerSimulator()
    
    # First we have to transpile the quantum circuit 
    # to the low-level QASM instructions used by the 
    # backend
    qc_compiled = transpile(qc, backend)
    
    # Execute the circuit on the qasm simulator.
    # We've set the number of repeats of the circuit
    # to be 1024, which is the default.
    job_sim = backend.run(qc_compiled, shots=1)
    
    # Grab the results from the job.
    result_sim = job_sim.result()
    
    counts = result_sim.get_counts(qc_compiled)
    print(counts)
    plot_histogram(counts)

# Things to do - 
# Make the functon fully depenedent on quantum registers and quantum gates
# make function consistent with the paper 
# reduce extra qubits used

# if this andGate implementation work then we can use it to implement the modular adder

# modularEcponentiation(a, x, n)
modularExponentiation(2, 2, 2)


m =  4
j =  0
i =  0
i + j =  0
j =  1
i =  0
i + j =  1
j =  0
i =  1
i + j =  1
j =  1
i =  1
i + j =  2
j =  0
i =  0
i + j =  0
j =  1
i =  0
i + j =  1
j =  2
i =  0
i + j =  2
j =  0
i =  1
i + j =  1
j =  1
i =  1
i + j =  2
j =  2
i =  1
i + j =  3
                         ░            ░       ░            ░      ┌───────── »
     control: ───────────░────────────░───────░────────────░──────┤          »
                         ░      ┌───┐ ░       ░            ░      │          »
   ancillary: ───────────░──|0>─┤ X ├─░───■───░────────■───░──────┤          »
                         ░      └─┬─┘ ░   │   ░ ┌───┐  │   ░ ┌───┐│          »
bitAncillary: ───────────░──|0>───┼───░───■───░─┤ X ├──■───░─┤ X ├┤          »
                         ░        │   ░ ┌─┴─┐ ░ └───┘  │   ░ └───┘│          »
         a_0: ───────────░────────┼───░─┤ X ├─░────────┼───░──────┤          »
                         ░        │   ░ └───┘ ░      ┌─┴─┐ ░      │          »
         a_1: ───────────░──────

CircuitTooWideForTarget: 'Number of qubits (31) in circuit-307 is greater than maximum (29) in the coupling_map'